## Python Code to Extract Data From Template and Transfer to PostGRE SQL
#### Authors : Aaron Liu, Rahul Venkatesh, Jessica Bonsu, Myeongyeon Lee 
##### Date Edited : 07-05-2023

In [5]:
## Required Packages
def required_packages():
    import pandas as pd
    import numpy as np
    import psycopg2 as pg
    import os
    from psycopg2.extras import Json
    from psycopg2.extensions import AsIs
    import functools
    import json
    import sys
    import requests
    import pprint
    
    print ('DONE') #remove when done
    

In [17]:
## Required Functions To Extract Information from Template
def read_all_sheets(filepath):
    def remove_emptyrows(df):
        nan_mask = ~df.iloc[:, 1].isna()
        return df[nan_mask]

    def split_df(df_):
        split_idx_mask = df_.iloc[:, 0].str.contains('#')
        w = df_[split_idx_mask].index.values
        df_list = []

        for i in range(len(w)-1):
            next_df = df_.loc[w[i]+1:w[i+1]-1, :]
            df_list.append(next_df)

        return df_list

    def table_to_dict(df_):
        main_mask = pd.isna(df_.JSON)
        step_dict = dict(df_[main_mask].iloc[:, :2].values)

        for json_field in pd.unique(df_.JSON):
            if pd.isna(json_field):
                continue
            elif json_field == 'data':
                data_mask = df_.JSON == 'data'
                step_dict['data'] = dict()

                for i, s in df_[data_mask].iterrows():
                    step_dict['data'][s[s.index[0]]] = s['value':'error_type'].dropna().to_dict()
            else:
                json_mask = df_.JSON == json_field
                step_dict[json_field] = dict(df_[json_mask].iloc[:, :2].values)

        return step_dict

    def read_sheet(filepath, sheet_name, ordering=False, usecols="A,B,D", meas=False):
        df = pd.read_excel(filepath, sheet_name=sheet_name, usecols=usecols)
        df_ = remove_emptyrows(df)
        sheet_dict = dict()

        if ordering == True:
            df_list = split_df(df_)
            for i, df in enumerate(df_list):
                sheet_dict[i] = table_to_dict(df)
        else:
            sheet_dict = table_to_dict(df_)
            
    #Storing each sheet in the template file as a dictionary
    exp_info = read_sheet(filepath, 'Data Origin')
    solution_makeup = read_sheet(filepath, 'Solution Makeup', ordering=True)
    solution_processing = read_sheet(filepath, 'Solution Treatment', ordering=True)
    device_fab = read_sheet(filepath, 'Device Fabrication')
    substrate_pretreat = read_sheet(filepath, 'Substrate Pretreat', ordering=True)
    coating_process = read_sheet(filepath, 'Coating Process')
    post_process = read_sheet(filepath, 'Post-Processing', ordering=True)
    device_meas = read_sheet(filepath, 'Device Measurement', usecols="A:G", ordering=True)
    other_meas = read_sheet(filepath, 'Other Measurements', usecols="A:G", ordering=True)

    return {
        'exp_info': exp_info,
        'solution_makeup': solution_makeup,
        'solution_processing': solution_processing,
        'device_fab': device_fab,
        'substrate_pretreat': substrate_pretreat,
        'coating_process': coating_process,
        'post_process': post_process,
        'device_meas': device_meas,
        'other_meas': other_meas
    }


### Reading and Extracting Data From Sheets in Template

In [18]:
fpath =r'..\db_feed\v6_example_blend.xlsx' 

result = read_all_sheets(fpath)

# Access the extracted information
exp_info = result['exp_info']
solution_makeup = result['solution_makeup']
solution_processing = result['solution_processing']
#....

C:\Users\jbonsu3\AppData\Local\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\jbonsu3\AppData\Local\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\jbonsu3\AppData\Local\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\jbonsu3\AppData\Local\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\jbonsu3\AppData\Local\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  fo

### Transferring Information From Template To PostgreSQL

In [ ]:
# Postgres python

# Adapters necessary for converting python data types to PostgreSQL compatible data types 
def register_pg_adapters():
    def addapt_numpy_float64(numpy_float64):
        return AsIs(numpy_float64)

    def addapt_numpy_int64(numpy_int64):
        return AsIs(numpy_int64)

    def nan_to_null(f, _NULL=AsIs('NULL'), _Float=pg.extensions.Float):
        if not np.isnan(f):
            return _Float(f)
        return _NULL

    pg.extensions.register_adapter(np.float64, addapt_numpy_float64)
    pg.extensions.register_adapter(np.int64, addapt_numpy_int64)
    pg.extensions.register_adapter(float, nan_to_null)


param_dict = {
    "host"      : "127.0.0.1",
    "database"  : "ofetdb_testenv_RV",
    "user"      : "postgres",
    "password"  : "Rahul2411!",
    "port"      : "5432",
}

def connect(params_dict):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = pg.connect(**params_dict)
    except (Exception, pg.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

def pg_query(sql, tup):
    
    try:
        # Database connection
        conn = connect(param_dict)
        cur = conn.cursor()
        
        
        
        # Pass SQL query, using string and placeholders
        cur.execute(sql, tup)
        
        # Fetch result
        fetched = cur.fetchone()[0]
        
        # Commit result
        conn.commit()
        print("Operation Successful")

        cur.close()
        conn.close()
        
    except (Exception, pg.DatabaseError) as error:
        # If database connection unsuccessful, then close connection 
        print("Error: %s" % error)
        conn.rollback()
        cur.close()
        conn.close()
    
    return fetched #return query result

def convert_entry(entry_dict):
    
    #This function reads a dictionary and extracts the column names and values from it
    
    pg_entry = entry_dict
    for key in pg_entry.keys():
        if type(pg_entry[key])==dict:
            pg_entry[key]=Json(pg_entry[key])
    columns = pg_entry.keys()
    values = [pg_entry[column] for column in columns]
    
    return pg_entry, columns, values

import requests
import json
import pandas as pd
import bibtexparser
from pprint import pprint


# Given a valid doi string, return a dictionary of digital object information.
def doi2dict(doi):
    url = "http://dx.doi.org/" + doi
    headers = {"accept": "application/x-bibtex"}
    r = requests.get(url, headers=headers).text
    bibdata = bibtexparser.bparser.BibTexParser().parse(r)
    return bibdata.entries[0]



In [ ]:
import json
import psycopg2
from psycopg2 import _json


### 1.Checking and Storing Experiment Information

# using the doi to extract additional information if citation type is 'literature

def store_experiment_info(exp_info, doi2dict):
    if exp_info['citation_type'] == 'literature':
        doi = exp_info['meta']['doi']
        doi_info = doi2dict(doi)  # Fetch additional information using DOI

        # Add the additional information to the existing dictionary
        exp_info['meta'].update(doi_info)

    exp_pg_entry, exp_columns, exp_values = convert_entry(exp_info)
    sql = '''
        INSERT INTO experiment_info (%s) 
        VALUES %s
        ON CONFLICT (citation_type, meta) DO UPDATE
        SET (%s) = %s
        RETURNING exp_id
        '''
    tup = (AsIs(','.join(exp_columns)), tuple(exp_values), AsIs(','.join(exp_columns)), tuple(exp_values))
    exp_id = pg_query(sql, tup)
    print("Experiment information stored successfully with id: {}".format(exp_id))


import psycopg2
import json

def store_solution_makeup(solution_makeup, param_dict):
    # Start transaction
    conn = psycopg2.connect(**param_dict)

    try:
        with conn:
            with conn.cursor() as cursor:
                # Storing Solvent data - accounting for multiple solvents
                solvent_data_filtered = [json_obj for json_obj in solution_makeup if json_obj.adapted.get("entity_type") == "solvent"]

                # Convert psycopg2._json.Json objects to JSON strings
                solvent_data = [json_obj.adapted for json_obj in solvent_data_filtered]

                # Storing Polymer data - accounting for multiple polymers
                polymer_data_filtered = [json_obj for json_obj in solution_makeup if json_obj.adapted.get("entity_type") == "polymer"]

                # Convert psycopg2._json.Json objects to JSON strings
                polymer_data = [json_obj.adapted for json_obj in polymer_data_filtered]

                # Storing Solution Makeup data
                solution_makeup_data = []
                solution_makeup_data.append(solution_makeup[0].adapted)
                solution_makeup_data.append(solvent_data)
                solution_makeup_data.append(polymer_data)

                # Extract solution information
                solution_data = solution_makeup_data[0]
                concentration = solution_data['concentration']

                # Extract solvent information
                solvent_data = solution_makeup_data[1]
                solvent_ids = []
                vol_fracs = []
                for solvent in solvent_data:
                    pubchem_cid = solvent['pubchem_cid']
                    iupac_name = solvent['iupac_name']
                    vol_frac = solvent['vol_frac']
                    solvent_ids.append((pubchem_cid, iupac_name))
                    vol_fracs.append(vol_frac)

                # Extract polymer information
                polymer_data = solution_makeup_data[2]
                polymer_ids = []
                wt_fracs = []
                for polymer in polymer_data:
                    common_name = polymer['common_name']
                    iupac_name = polymer['iupac_name']
                    mn = polymer['mn']
                    mw = polymer['mw']
                    dispersity = polymer['dispersity']
                    wt_frac = polymer['wt_frac']
                    meta = json.dumps(polymer['meta'])
                    polymer_ids.append((common_name, iupac_name, mn, mw, dispersity, meta))
                    wt_fracs.append(wt_frac)

                # Check if the unique combination exists
                select_solution_id_sql = '''
                    SELECT sm.solution_id
                    FROM SOLUTION_MAKEUP_SOLVENT sms
                    JOIN SOLUTION_MAKEUP_POLYMER smp ON sms.solution_id = smp.solution_id
                    JOIN SOLVENT s ON sms.solvent_id = s.pubchem_cid
                    JOIN POLYMER p ON smp.polymer_id = p.polymer_id
                    JOIN SOLUTION sm ON sms.solution_id = sm.solution_id
                    WHERE sm.concentration = %s
                    AND (s.pubchem_cid, s.iupac_name) IN %s
                    AND (p.common_name, p.iupac_name, p.mn, p.mw, p.dispersity, p.meta) IN %s
                    GROUP BY sm.solution_id
                    HAVING COUNT(DISTINCT smp.polymer_id) = %s
                    AND COUNT(DISTINCT sms.solvent_id) = %s
                    AND ARRAY_AGG(sms.vol_frac) = %s::double precision[]
                    AND ARRAY_AGG(smp.wt_frac) = %s::double precision[]
                '''

                cursor.execute(select_solution_id_sql, (concentration, tuple(solvent_ids), tuple(polymer_ids), len(polymer_ids), len(solvent_ids), vol_fracs, wt_fracs))
                existing_solution = cursor.fetchone()

                #Checking if there is an existing solution
                if existing_solution:
                    solution_id = existing_solution[0]
                else:
                    # Insert into SOLUTION table
                    insert_solution_sql = '''
                        INSERT INTO SOLUTION (concentration)
                        VALUES (%s)
                        RETURNING solution_id
                    '''
                    cursor.execute(insert_solution_sql, (concentration,))
                    solution_id = cursor.fetchone()[0]

                # Read Solvent data
                for solvent_id, vol_frac in zip(solvent_ids, vol_fracs):
                    pubchem_cid, iupac_name = solvent_id

                    # Check if the solvent exists
                    select_solvent_id_sql = '''
                        SELECT pubchem_cid
                        FROM SOLVENT
                        WHERE iupac_name = %s
                    '''
                    cursor.execute(select_solvent_id_sql, (iupac_name,))
                    existing_solvent = cursor.fetchone()

                    if existing_solvent:
                        solvent_id = existing_solvent[0]
                    else:
                        # Insert into SOLVENT table
                        insert_solvent_sql = '''
                            INSERT INTO SOLVENT (pubchem_cid, iupac_name)
                            VALUES (%s, %s)
                            RETURNING pubchem_cid
                        '''
                        cursor.execute(insert_solvent_sql, (pubchem_cid, iupac_name))
                        solvent_id = cursor.fetchone()[0]

                    # Insert or update SOLUTION_MAKEUP_SOLVENT table
                    insert_solution_makeup_solvent_sql = '''
                        INSERT INTO SOLUTION_MAKEUP_SOLVENT (solution_id, solvent_id, vol_frac)
                        VALUES (%s, %s, %s)
                        ON CONFLICT (solution_id, solvent_id, vol_frac) DO UPDATE
                        SET solution_id = SOLUTION_MAKEUP_SOLVENT.solution_id,
                            solvent_id = SOLUTION_MAKEUP_SOLVENT.solvent_id,
                            vol_frac = SOLUTION_MAKEUP_SOLVENT.vol_frac
                    '''
                    cursor.execute(insert_solution_makeup_solvent_sql, (solution_id, solvent_id, vol_frac))

                # Read the polymer data
                for polymer_id, wt_frac in zip(polymer_ids, wt_fracs):
                    common_name, iupac_name, mn, mw, dispersity, meta = polymer_id

                    # Check if the polymer exists
                    select_polymer_id_sql = '''
                        SELECT polymer_id
                        FROM POLYMER
                        WHERE common_name = %s
                        AND iupac_name = %s
                        AND mn = %s
                        AND mw = %s
                        AND dispersity = %s
                        AND meta = %s::jsonb
                    '''
                    cursor.execute(select_polymer_id_sql, (common_name, iupac_name, mn, mw, dispersity, meta))
                    existing_polymer = cursor.fetchone()

                    if existing_polymer:
                        polymer_id = existing_polymer[0]
                    else:
                        # Insert into POLYMER table
                        insert_polymer_sql = '''
                            INSERT INTO POLYMER (common_name, iupac_name, mn, mw, dispersity, meta)
                            VALUES (%s, %s, %s, %s, %s, %s::jsonb)
                            RETURNING polymer_id
                        '''
                        cursor.execute(insert_polymer_sql, (common_name, iupac_name, mn, mw, dispersity, meta))
                        polymer_id = cursor.fetchone()[0]

                    # Insert or update SOLUTION_MAKEUP_POLYMER table
                    insert_solution_makeup_polymer_sql = '''
                        INSERT INTO SOLUTION_MAKEUP_POLYMER (solution_id, polymer_id, wt_frac)
                        VALUES (%s, %s, %s)
                        ON CONFLICT (solution_id, polymer_id, wt_frac) DO UPDATE
                        SET solution_id = SOLUTION_MAKEUP_POLYMER.solution_id,
                            polymer_id = SOLUTION_MAKEUP_POLYMER.polymer_id,
                            wt_frac = SOLUTION_MAKEUP_POLYMER.wt_frac
                    '''
                    cursor.execute(insert_solution_makeup_polymer_sql, (solution_id, polymer_id, wt_frac))

                print("Solution makeup information saved successfully with id : {}".format(solution_id))

    except Exception as e:
        print("An error occurred:", str(e))

    # Close the database connection
    conn.close()

# Usage example:
store_solution_makeup(solution_makeup, param_dict)

import psycopg2
import json

def store_device_fabrication(device_fab, param_dict):
    # Start transaction
    conn = psycopg2.connect(**param_dict)

    try:
        with conn:
            with conn.cursor() as cursor:
                # Convert device fabrication entry to columns and values
                device_fab_pg_entry, device_fab_columns, device_fab_values = convert_entry(device_fab)

                # If meta information is missing, add it as an empty dictionary
                device_fab_columns_list = list(device_fab_columns)
                if 'meta' not in device_fab_columns_list:
                    device_fab_columns_list.append('meta')
                    device_fab_values.append({})

                # Convert values to JSON strings if they are dictionaries
                device_fab_values = [json.dumps(value) if isinstance(value, dict) else value for value in device_fab_values]

                # Insert into DEVICE_FABRICATION table
                sql = '''
                    INSERT INTO DEVICE_FABRICATION (%s) 
                    VALUES %s
                    ON CONFLICT (params, meta) DO UPDATE
                    SET (%s) = %s
                    RETURNING device_fab_id
                '''
                tup = (AsIs(','.join(device_fab_columns_list)), tuple(device_fab_values), AsIs(','.join(device_fab_columns_list)), tuple(device_fab_values))
                cursor.execute(sql, tup)
                device_fab_id = cursor.fetchone()[0]

                print("Device fabrication information saved successfully with id: {}".format(device_fab_id))

    except Exception as e:
        print("An error occurred:", str(e))

    # Close the database connection
    conn.close()

# Usage example:
store_device_fabrication(device_fab, param_dict)
import psycopg2
import json

def store_film_deposition(coating_process, param_dict):
    # Start transaction
    conn = psycopg2.connect(**param_dict)

    try:
        with conn:
            with conn.cursor() as cursor:
                # Convert coating process entry to columns and values
                coating_process_pg_entry, coating_process_columns, coating_process_values = convert_entry(coating_process)

                # If meta information is missing, add it as an empty dictionary
                coating_process_columns_list = list(coating_process_columns)
                if 'meta' not in coating_process_columns_list:
                    coating_process_columns_list.append('meta')
                    coating_process_values.append({})

                # Convert values to JSON strings if they are dictionaries
                coating_process_values = [json.dumps(value) if isinstance(value, dict) else value for value in coating_process_values]

                # Insert into FILM_DEPOSITION table
                sql = '''
                    INSERT INTO FILM_DEPOSITION (%s) 
                    VALUES %s
                    ON CONFLICT (deposition_type, params, meta) DO UPDATE
                    SET (%s) = %s
                    RETURNING film_deposition_id
                '''
                tup = (AsIs(','.join(coating_process_columns_list)), tuple(coating_process_values), AsIs(','.join(coating_process_columns_list)), tuple(coating_process_values))
                cursor.execute(sql, tup)
                film_deposition_id = cursor.fetchone()[0]

                print("Film deposition information saved successfully with id: {}".format(film_deposition_id))

    except Exception as e:
        print("An error occurred:", str(e))

    # Close the database connection
    conn.close()

# Usage example:
store_film_deposition(coating_process, param_dict)

### 5. Checking and Storing the subprocess recipes (Solution Treatment, Substrate Pretreatment, Post Process)
#### 5.1 SOLUTION TREATMENT

def store_solution_treatment(param_dict, solution_processing):
    # Function to insert data into SOLUTION_TREATMENT_STEP table
    def insert_into_solution_treatment_step(cur, treatment_type, params, meta):
        # Check if the record already exists
        cur.execute(
            "SELECT solution_treatment_step_id FROM SOLUTION_TREATMENT_STEP WHERE treatment_type = %s AND params = %s::jsonb AND meta = %s::jsonb",
            (treatment_type, params, meta)
        )
        existing_id = cur.fetchone()

        if existing_id:
            solution_treatment_step_id = existing_id[0]
        else:
            # Insert data into SOLUTION_TREATMENT_STEP table
            cur.execute(
                "INSERT INTO SOLUTION_TREATMENT_STEP (treatment_type, params, meta) VALUES (%s, %s::jsonb, %s::jsonb) RETURNING solution_treatment_step_id",
                (treatment_type, params, meta)
            )
            solution_treatment_step_id = cur.fetchone()[0]

        return solution_treatment_step_id

    # Function to insert data into SOLUTION_TREATMENT_ORDER table
    def insert_into_solution_treatment_order(cur, solution_treatment_id, process_order, solution_treatment_step_id):
        # Check if the record already exists
        cur.execute(
            """
            SELECT solution_treatment_id
            FROM SOLUTION_TREATMENT_ORDER
            WHERE solution_treatment_id = %s
            AND process_order = %s
            AND solution_treatment_step_id = %s
            """,
            (solution_treatment_id, process_order, solution_treatment_step_id)
        )
        existing_combination = cur.fetchone()

        if not existing_combination:
            # Insert new record into SOLUTION_TREATMENT_ORDER table
            cur.execute(
                "INSERT INTO SOLUTION_TREATMENT_ORDER (solution_treatment_id, process_order, solution_treatment_step_id) VALUES (%s, %s, %s)",
                (solution_treatment_id, process_order, solution_treatment_step_id)
            )

        return solution_treatment_id

    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(**param_dict)

    # Create a cursor object to interact with the database
    cur = conn.cursor()

    solution_treatment_id = None

    for treatment in solution_processing.values():
        # Convert params and meta to JSON format
        params_json = json.dumps(treatment.get('params', {}))
        meta_json = json.dumps(treatment.get('meta', {}))

        # Insert data into SOLUTION_TREATMENT_STEP table
        solution_treatment_step_id = insert_into_solution_treatment_step(cur, treatment['treatment_type'], params_json, meta_json)

        if solution_treatment_id is None:
            # Check if the record already exists in SOLUTION_TREATMENT table
            cur.execute(
                "SELECT solution_treatment_id FROM SOLUTION_TREATMENT WHERE solution_treatment_id IN (SELECT solution_treatment_id FROM SOLUTION_TREATMENT_ORDER WHERE solution_treatment_step_id = %s)",
                (solution_treatment_step_id,)
            )
            existing_id = cur.fetchone()

            if existing_id:
                solution_treatment_id = existing_id[0]
            else:
                # Insert data into SOLUTION_TREATMENT table
                cur.execute(
                    "INSERT INTO SOLUTION_TREATMENT (solution_treatment_id) VALUES (DEFAULT) RETURNING solution_treatment_id"
                )
                solution_treatment_id = cur.fetchone()[0]

        # Insert data into SOLUTION_TREATMENT_ORDER table
        solution_treatment_id = insert_into_solution_treatment_order(cur, solution_treatment_id, treatment['process_step'], solution_treatment_step_id)

    # Commit the changes to the database
    print("Solution treatment saved successfully with id : {}".format(solution_treatment_id))
    conn.commit()

    # Close the cursor and connection
    cur.close()
    conn.close()


###### 5.2 SUBSTRATE PRETREATMENT

# Function to insert data into SUBSTRATE_PRETREAT_STEP table
import psycopg2
import json

def store_substrate_pretreatment(param_dict, substrate_pretreat):
    # Function to insert data into SUBSTRATE_PRETREAT_STEP table
    def insert_into_substrate_pretreat_step(cur, treatment_type, params, meta):
        # Check if the record already exists
        cur.execute(
            "SELECT substrate_pretreat_step_id FROM SUBSTRATE_PRETREAT_STEP WHERE treatment_type = %s AND params = %s::jsonb AND meta = %s::jsonb",
            (treatment_type, params, meta)
        )
        existing_id = cur.fetchone()

        if existing_id:
            substrate_pretreat_step_id = existing_id[0]
        else:
            # Insert data into SUBSTRATE_PRETREAT_STEP table
            cur.execute(
                "INSERT INTO SUBSTRATE_PRETREAT_STEP (treatment_type, params, meta) VALUES (%s, %s::jsonb, %s::jsonb) RETURNING substrate_pretreat_step_id",
                (treatment_type, params, meta)
            )
            substrate_pretreat_step_id = cur.fetchone()[0]

        return substrate_pretreat_step_id

    # Function to insert data into SUBSTRATE_PRETREAT_ORDER table
    def insert_into_substrate_pretreat_order(cur, substrate_pretreat_id, process_order, substrate_pretreat_step_id):
        # Check if the record already exists
        cur.execute(
            """
            SELECT substrate_pretreat_id
            FROM SUBSTRATE_PRETREAT_ORDER
            WHERE substrate_pretreat_id = %s
            AND process_order = %s
            AND substrate_pretreat_step_id = %s
            """,
            (substrate_pretreat_id, process_order, substrate_pretreat_step_id)
        )
        existing_combination = cur.fetchone()

        if not existing_combination:
            # Insert new record into SUBSTRATE_PRETREAT_ORDER table
            cur.execute(
                "INSERT INTO SUBSTRATE_PRETREAT_ORDER (substrate_pretreat_id, process_order, substrate_pretreat_step_id) VALUES (%s, %s, %s)",
                (substrate_pretreat_id, process_order, substrate_pretreat_step_id)
            )

        return substrate_pretreat_id

    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(**param_dict)

    # Create a cursor object to interact with the database
    cur = conn.cursor()

    substrate_pretreat_id = None

    for treatment in substrate_pretreat.values():
        # Convert params and meta to JSON format
        params_json = json.dumps(treatment.get('params', {}))
        meta_json = json.dumps(treatment.get('meta', {}))

        # Insert data into SUBSTRATE_PRETREAT_STEP table
        substrate_pretreat_step_id = insert_into_substrate_pretreat_step(cur, treatment['treatment_type'], params_json, meta_json)

        if substrate_pretreat_id is None:
            # Check if the record already exists in SUBSTRATE_PRETREAT table
            cur.execute(
                "SELECT substrate_pretreat_id FROM SUBSTRATE_PRETREAT WHERE substrate_pretreat_id IN (SELECT substrate_pretreat_id FROM SUBSTRATE_PRETREAT_ORDER WHERE substrate_pretreat_step_id = %s)",
                (substrate_pretreat_step_id,)
            )
            existing_id = cur.fetchone()

            if existing_id:
                substrate_pretreat_id = existing_id[0]
            else:
                # Insert data into SUBSTRATE_PRETREAT table
                cur.execute(
                    "INSERT INTO SUBSTRATE_PRETREAT (substrate_pretreat_id) VALUES (DEFAULT) RETURNING substrate_pretreat_id"
                )
                substrate_pretreat_id = cur.fetchone()[0]

        # Insert data into SUBSTRATE_PRETREAT_ORDER table
        substrate_pretreat_id = insert_into_substrate_pretreat_order(cur, substrate_pretreat_id, treatment['process_step'], substrate_pretreat_step_id)

    # Commit the changes to the database
    print("Substrate pretreatment saved successfully with id : {}".format(substrate_pretreat_id))
    conn.commit()

    # Close the cursor and connection
    cur.close()
    conn.close()


###### 5.3 POST PROCESSING TREATMENT
# Function to insert data into POSTPROCESS_STEP table
def insert_into_postprocess_step(cur, treatment_type, params, meta):
    # Check if the record already exists
    cur.execute(
        "SELECT postprocess_step_id FROM POSTPROCESS_STEP WHERE treatment_type = %s AND params = %s::jsonb AND meta = %s::jsonb",
        (treatment_type, params, meta)
    )
    existing_id = cur.fetchone()

    if existing_id:
        postprocess_step_id = existing_id[0]
    else:
        # Insert data into POSTPROCESS_STEP table
        cur.execute(
            "INSERT INTO POSTPROCESS_STEP (treatment_type, params, meta) VALUES (%s, %s::jsonb, %s::jsonb) RETURNING postprocess_step_id",
            (treatment_type, params, meta)
        )
        postprocess_step_id = cur.fetchone()[0]

    return postprocess_step_id

# Function to insert data into POSTPROCESS_ORDER table
def insert_into_postprocess_order(cur, postprocess_id, process_order, postprocess_step_id):
    # Check if the record already exists
    cur.execute(
        """
        SELECT postprocess_id
        FROM POSTPROCESS_ORDER
        WHERE postprocess_id = %s
        AND process_order = %s
        AND postprocess_step_id = %s
        """,
        (postprocess_id, process_order, postprocess_step_id)
    )
    existing_combination = cur.fetchone()

    if not existing_combination:
        # Insert new record into POSTPROCESS_ORDER table
        cur.execute(
            "INSERT INTO POSTPROCESS_ORDER (postprocess_id, process_order, postprocess_step_id) VALUES (%s, %s, %s)",
            (postprocess_id, process_order, postprocess_step_id)
        )

    return postprocess_id

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(**param_dict)

# Create a cursor object to interact with the database
cur = conn.cursor()

postprocess_id = None

for treatment in post_process.values():
    # Convert params and meta to JSON format
    params_json = json.dumps(treatment.get('params', {}))
    meta_json = json.dumps(treatment.get('meta', {}))

    # Insert data into POSTPROCESS_STEP table
    postprocess_step_id = insert_into_postprocess_step(cur, treatment['treatment_type'], params_json, meta_json)

    if postprocess_id is None:
        # Check if the record already exists in POSTPROCESS table
        cur.execute(
            "SELECT postprocess_id FROM POSTPROCESS WHERE postprocess_id IN (SELECT postprocess_id FROM POSTPROCESS_ORDER WHERE postprocess_step_id = %s)",
            (postprocess_step_id,)
        )
        existing_id = cur.fetchone()

        if existing_id:
            postprocess_id = existing_id[0]
        else:
            # Insert data into POSTPROCESS table
            cur.execute(
                "INSERT INTO POSTPROCESS (postprocess_id) VALUES (DEFAULT) RETURNING postprocess_id"
            )
            postprocess_id = cur.fetchone()[0]

    # Insert data into POSTPROCESS_ORDER table
    postprocess_id = insert_into_postprocess_order(cur, postprocess_id, treatment['process_step'], postprocess_step_id)

# Commit the changes to the database
print("Post Process treatment saved successfully with id : {}".format(postprocess_id))
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

### 6. Checking and Storing information to the OFET_PROCESS TABLE and generating process_id
import psycopg2
import json

def store_post_process_treatment(param_dict, post_process):
    # Function to insert data into POSTPROCESS_STEP table
    def insert_into_postprocess_step(cur, treatment_type, params, meta):
        # Check if the record already exists
        cur.execute(
            "SELECT postprocess_step_id FROM POSTPROCESS_STEP WHERE treatment_type = %s AND params = %s::jsonb AND meta = %s::jsonb",
            (treatment_type, params, meta)
        )
        existing_id = cur.fetchone()

        if existing_id:
            postprocess_step_id = existing_id[0]
        else:
            # Insert data into POSTPROCESS_STEP table
            cur.execute(
                "INSERT INTO POSTPROCESS_STEP (treatment_type, params, meta) VALUES (%s, %s::jsonb, %s::jsonb) RETURNING postprocess_step_id",
                (treatment_type, params, meta)
            )
            postprocess_step_id = cur.fetchone()[0]

        return postprocess_step_id

    # Function to insert data into POSTPROCESS_ORDER table
    def insert_into_postprocess_order(cur, postprocess_id, process_order, postprocess_step_id):
        # Check if the record already exists
        cur.execute(
            """
            SELECT postprocess_id
            FROM POSTPROCESS_ORDER
            WHERE postprocess_id = %s
            AND process_order = %s
            AND postprocess_step_id = %s
            """,
            (postprocess_id, process_order, postprocess_step_id)
        )
        existing_combination = cur.fetchone()

        if not existing_combination:
            # Insert new record into POSTPROCESS_ORDER table
            cur.execute(
                "INSERT INTO POSTPROCESS_ORDER (postprocess_id, process_order, postprocess_step_id) VALUES (%s, %s, %s)",
                (postprocess_id, process_order, postprocess_step_id)
            )

        return postprocess_id

    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(**param_dict)

    # Create a cursor object to interact with the database
    cur = conn.cursor()

    postprocess_id = None

    for treatment in post_process.values():
        # Convert params and meta to JSON format
        params_json = json.dumps(treatment.get('params', {}))
        meta_json = json.dumps(treatment.get('meta', {}))

        # Insert data into POSTPROCESS_STEP table
        postprocess_step_id = insert_into_postprocess_step(cur, treatment['treatment_type'], params_json, meta_json)

        if postprocess_id is None:
            # Check if the record already exists in POSTPROCESS table
            cur.execute(
                "SELECT postprocess_id FROM POSTPROCESS WHERE postprocess_id IN (SELECT postprocess_id FROM POSTPROCESS_ORDER WHERE postprocess_step_id = %s)",
                (postprocess_step_id,)
            )
            existing_id = cur.fetchone()

            if existing_id:
                postprocess_id = existing_id[0]
            else:
                # Insert data into POSTPROCESS table
                cur.execute(
                    "INSERT INTO POSTPROCESS (postprocess_id) VALUES (DEFAULT) RETURNING postprocess_id"
                )
                postprocess_id = cur.fetchone()[0]

        # Insert data into POSTPROCESS_ORDER table
        postprocess_id = insert_into_postprocess_order(cur, postprocess_id, treatment['process_step'], postprocess_step_id)

    # Commit the changes to the database
    print("Post Process treatment saved successfully with id : {}".format(postprocess_id))
    conn.commit()

    # Close the cursor and connection
    cur.close()
    conn.close()


### 7. Checking and Storing information to the SAMPLE TABLE and generating sample_id
def insert_into_sample_table(pg_query, sample_columns, sample_values):
    sql = '''
        INSERT INTO sample (%s) 
        VALUES %s
        ON CONFLICT (exp_id, process_id) DO UPDATE
        SET (%s) = %s
        RETURNING sample_id
        '''

    tup = (AsIs(','.join(sample_columns)), tuple(sample_values), AsIs(','.join(sample_columns)), tuple(sample_values))

    sample_id = pg_query(sql, tup)
    print("sample_id is: {}".format(sample_id))

# Usage
sample_columns = ['exp_id', 'process_id']
sample_values = [exp_id, process_id]

insert_into_sample_table(pg_query, sample_columns, sample_values)

### 8. Checking and Storing the measurement information 
#### 8.1 Storing Device Measurement Information 
def insert_into_measurement_table(pg_query, sample_id, device_meas_columns, device_meas_values):
    device_meas_columns_list = list(device_meas_columns)  # Convert dict_keys to a list
    device_meas_columns_list.insert(0, 'sample_id')
    device_meas_values.insert(0, sample_id)

    # If meta information is missing
    if 'meta' not in device_meas_columns_list:
        device_meas_columns_list.append('meta')
        device_meas_values.append({})

    device_meas_values = [json.dumps(value) if isinstance(value, dict) else value for value in device_meas_values]

    sql = '''
        INSERT INTO measurement (%s) 
        VALUES %s
        ON CONFLICT (sample_id, measurement_type, data, meta) DO UPDATE
        SET (%s) = %s
        RETURNING measurement_id
        '''

    tup = (
        AsIs(','.join(device_meas_columns_list)),
        tuple(device_meas_values),
        AsIs(','.join(device_meas_columns_list)),
        tuple(device_meas_values)
    )

    measurement_id = pg_query(sql, tup)
    print("device measurement information saved successfully with id: {}".format(measurement_id))

# Usage
sample_id = ...  # Provide the sample_id value
device_meas_columns = device_meas_pg_entry
device_meas_values = device_meas_values

insert_into_measurement_table(pg_query, sample_id, device_meas_columns, device_meas_values)


#### 8.2 Storing Other Measurement Information 
def insert_other_meas_into_measurement_table(pg_query, sample_id, other_meas):
    for items in other_meas:
        other_meas_pg_entry, other_meas_columns, other_meas_values = convert_entry(other_meas[items])

        other_meas_columns_list = list(other_meas_columns)  # Convert dict_keys to a list
        other_meas_columns_list.insert(0, 'sample_id')
        other_meas_values.insert(0, sample_id)

        # If meta information is missing
        if 'meta' not in other_meas_columns_list:
            other_meas_columns_list.append('meta')
            other_meas_values.append({})

        other_meas_values = [json.dumps(value) if isinstance(value, dict) else value for value in other_meas_values]

        sql = '''
        INSERT INTO measurement (%s) 
        VALUES %s
        ON CONFLICT (sample_id, measurement_type, data, meta) DO UPDATE
        SET (%s) = %s
        RETURNING measurement_id
        '''

        tup = (
            AsIs(','.join(other_meas_columns_list)),
            tuple(other_meas_values),
            AsIs(','.join(other_meas_columns_list)),
            tuple(other_meas_values)
        )

        measurement_id = pg_query(sql, tup)
        print("other measurement information saved successfully with id: {}".format(measurement_id))

# Usage
sample_id = ...  # Provide the sample_id value
other_meas = ...  # Provide the other_meas dictionary

insert_other_meas_into_measurement_table(pg_query, sample_id, other_meas)


print("Record stored successfully")